# Faster search

https://github.com/facebookresearch/faiss/wiki/Faster-search

用numpy生成0-1均匀分布，它们的值在`(0,1)`范围内。为了增加娱乐性，我们在第一个向量上加个小平移。

`np.random.seed(1234)`可以保证每次numpy生成的测试数据都是一样的。

In [6]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [7]:
print('xb.shape', xb.shape)
print('xb[:1]', xb[:1])
print('xq.shape', xq.shape)
print('xq[:1]', xq[:1])

xb.shape (100000, 64)
xb[:1] [[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926
  0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951
  0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845
  0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824
  0.9331401  0.65137815 0.39720258 0.78873014 0.31683612 0.56809866
  0.8691274  0.4361734  0.8021476  0.14376682 0.70426095 0.7045813
  0.21879211 0.92486763 0.44214076 0.90931594 0.05980922 0.18428709
  0.04735528 0.6748809  0.59462476 0.5333102  0.04332406 0.5614331
  0.32966843 0.5029668  0.11189432 0.6071937  0.5659447  0.00676406
  0.6174417  0.9121229  0.7905241  0.99208146 0.95880175 0.7919641
  0.28525096 0.62491673 0.4780938  0.19567518]]
xq.shape (10000, 64)
xq[:1] [[0.81432974 0.7409969  0.8915324  0.02642949 0.24954738 0.75948536
  0.33756447 0.0388501  0.06253924 0.04496585 0.6500265  0.14300306
  0.10555115 0.7554373  0.8733019  0.91065574 0.949595   0.4678057
  0.7957018  0.0608

## This is too slow, how can I make it faster?——IndexIVFFlat

为了加快搜索，faiss将数据集分割成块. faiss在`d`维空间中定义`Voronoi cells`, 语料中的每个向量都会落在其中一个cell. 当搜索时, 只将查询向量`x`所在的cell中包含的数据库向量`y`进行比较，只将很少的近邻与查询向量比较.

> 如果查询向量`x`与数据库向量`y`没有落在同一个cell，那就没必要比较，这样就加快了搜索速度。

这是通过`IndexIVFFlat`索引完成的. 该索引需要一个训练阶段, 可以在具有相同分布（distribution）的任意数据库向量集合上执行. In this case we just use the database vectors themselves.

`IndexIVFFlat` 也需要另一个索引——quantizer（量化器），把向量分配到`Voronoi cells`. 每个cell是由`centroid`定义的, 要查找一个向量落在哪个`Voronoi cell`, 需要在centroid集合中查找该向量的最近邻. 这个任务由另一个索引完成，通常情况是`IndexFlatL2`.

有两个参数：
- `nlist`, cell的数量
- `nprobe`, 执行搜索时要访问cell的数量

搜索时间大致随`nprobe`数量的增加而线性增加（再加上量化后的某个常数）。当`nprobe=nlist`时，相当于暴力搜索。

In [11]:
nlist = 100
k = 4

import faiss 

quantizer = faiss.IndexFlatL2(d)  # the other index as 
index = faiss.IndexIVFFlat(quantizer, d, nlist)
print(index.is_trained)
index.train(xb)
print(index.is_trained)

index.add(xb) # add may be a bit slower as well

False
True


In [12]:
D, I = index.search(xq, k)     # actual search
print(len(I))                  # len(I) is equal to nq
print(I[-5:])                  # neighbors of the 5 last queries

10000
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]


In [13]:
index.nprobe = 10              # default nprobe is 1, try a few more
D, I = index.search(xq, k)
print(len(I))
print(I[-5:])                  # neighbors of the 5 last queries

10000
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [14]:
index.nprobe = nlist           # default nprobe is 1, try a few more
D, I = index.search(xq, k)
print(len(I))
print(I[-5:])                  # neighbors of the 5 last queries

10000
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


备注：
- 你会发现，上面三个search的速度和准确性，差异很大，因为设置了不同nprobe值

# Results

先看看暴力搜索结果, 也即是第三次的搜索。`（衡量搜索的准确性，与暴力搜索相比。）`。
```text
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
```

当 `nprobe=1`时, 结果：
```text
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]
```
该结果与暴力搜索相比有区别，不够准确。
This is because some of the results were not in the exact same Voronoi cell. Therefore, visiting a few more cells may prove useful.

当 nprobe 增加到 `10`或`100`时，结果：
```text
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
```
这是正确的. 注意，上面，获得了一个完美的结果，这是因为数据分布是人为干预了，实际情况肯定不会这么顺利。

`nprobe` 参数是调节速度与准确性之间的折衷方法. 

当设置 `nprobe=nlist` 时，等价于暴力搜索（brute-force search），此时会搜索会变慢.

# Results

先看看暴力搜索结果, 也即是第三次的搜索。`（衡量搜索的准确性，与暴力搜索相比。）`。
```text
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
```

当 `nprobe=1`时, 结果：
```text
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]
```
该结果与暴力搜索相比有区别，不够准确。
This is because some of the results were not in the exact same Voronoi cell. Therefore, visiting a few more cells may prove useful.

当 nprobe 增加到 `10`或`100`时，结果：
```text
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
```
这是正确的. 注意，上面，获得了一个完美的结果，这是因为数据分布是人为干预了，实际情况肯定不会这么顺利。

`nprobe` 参数是调节速度与准确性之间的折衷方法. 

当设置 `nprobe=nlist` 时，等价于暴力搜索（brute-force search），此时会搜索会变慢.